<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-3">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-4">Main</a></span><ul class="toc-item"><li><span><a href="#CBTN" data-toc-modified-id="CBTN-4.1">CBTN</a></span></li><li><span><a href="#ICGC" data-toc-modified-id="ICGC-4.2">ICGC</a></span></li><li><span><a href="#TARGET" data-toc-modified-id="TARGET-4.3">TARGET</a></span></li><li><span><a href="#TCGA" data-toc-modified-id="TCGA-4.4">TCGA</a></span></li><li><span><a href="#Merge-all" data-toc-modified-id="Merge-all-4.5">Merge all</a></span></li></ul></li></ul></div>

# Dependencies

In [1]:
library(tidyr)
library(biomaRt)

# Functions

In [2]:
matchexp_matrixfx <- function(exp_matrix, estimate_manifest, group, sample_type, matchingcol){
    
    message("dimensions of exp_matrix: ", deparse(substitute(exp_matrix)))
    print(dim(exp_matrix)) 
    subset_estimate_manifest <- estimate_manifest[estimate_manifest$group == group,]
    subset_estimate_manifest_sampletype <- subset_estimate_manifest[grepl(sample_type, subset_estimate_manifest$sample_type),]
    message("dimensions of estimate_manifest_df for: ", group, " sample_type:", sample_type)
    print(dim(subset_estimate_manifest_sampletype)) 
    exp_matrix_matchingset <- exp_matrix[,colnames(exp_matrix) %in% subset_estimate_manifest_sampletype[[matchingcol]]] 
    message("dimensions of exp_matrix_matchingset for: ", deparse(substitute(exp_matrix)), " with matching column: ", matchingcol)
    print(dim(exp_matrix_matchingset))
    exp_matrix_matchingset <- cbind(exp_matrix[,1:2], exp_matrix_matchingset)
    
    return(exp_matrix_matchingset)
}

# Paths

In [3]:
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Manifests/"
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/IPD2/Plots/"

# Main

Subset to primary samples

## CBTN

This file is post cleanup, post estimate runs, post primary subset, pre clustering. for expression_clustering

In [4]:
CBTTCtpm <- read.table(paste0(datapath, "exp_mat/CBTTC_tpm_matrix_dedup.txt"), header = T, sep = "\t", stringsAsFactors = F, check.names = F)

In [5]:
CBTTCtpm[1:10,1:10]

,ensembl_id,hgnc_symbol,7316-2599,7316-1926,7316-489,7316-16,7316-13,7316-3936,7316-3632,7316-873
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000000003.14,TSPAN6,10.99,8.66,6.15,2.40,0.73,1.50,1.88,29.41
2,ENSG00000000005.5,TNMD,0.18,0.02,0.03,0.02,0.02,0.01,0.03,0.33
3,ENSG00000000419.12,DPM1,4.95,6.15,5.09,3.35,1.09,3.68,3.27,4.89
4,ENSG00000000457.13,SCYL3,1.50,1.75,1.71,0.90,0.44,1.44,0.89,1.52
5,ENSG00000000460.16,C1orf112,1.70,1.12,2.78,0.34,0.43,0.71,0.57,1.00
6,ENSG00000000938.12,FGR,1.27,0.90,0.18,1.12,0.36,1.32,1.05,0.93
7,ENSG00000000971.15,CFH,2.04,5.77,2.02,7.50,5.57,1.02,13.23,2.75
8,ENSG00000001036.13,FUCA2,7.86,7.15,3.33,4.47,1.40,1.88,8.94,10.26
9,ENSG00000001084.10,GCLC,14.26,13.19,7.39,11.16,1.20,15.90,3.18,16.94


In [6]:
load(file = paste0(datapath, "/ESTIMATE/estimate_manifest_primary_clean.RData"))

In [7]:
table(estimate_manifest_primary_clean$group)


  CBTN   ICGC TARGET   TCGA 
   632    196    149   2452 

In [8]:
CBTTC_tpm_matrix_primary <- matchexp_matrixfx(CBTTCtpm, estimate_manifest_primary_clean,"CBTN", "Initial", "sample_id")

dimensions of exp_matrix: CBTTCtpm



[1] 56505   971


dimensions of estimate_manifest_df for: CBTN sample_type:Initial



[1] 632  21


dimensions of exp_matrix_matchingset for: CBTTCtpm with matching column: sample_id



[1] 56505   632


In [9]:
CBTTC_tpm_matrix_primary$ensembl_id <- gsub("[.].*", "", CBTTC_tpm_matrix_primary$ensembl_id)

In [10]:
CBTN_tpm_matrix_primary_dedup <- CBTTC_tpm_matrix_primary[!duplicated(CBTTC_tpm_matrix_primary$ensembl_id),]

In [11]:
dim(CBTTC_tpm_matrix_primary)
dim(CBTN_tpm_matrix_primary_dedup)

[1] 56505   634

[1] 56505   634

In [12]:
write.table(CBTN_tpm_matrix_primary_dedup,
            file = paste0(datapath, "exp_mat/CBTTC_tpm_matrix_primary_dedup.txt"), sep = "\t", quote = T, row.names = F)

## ICGC

In [13]:
ICGC <- read.table(file = paste0(datapath,"exp_mat/ICGC.tpm.matrix.txt"),
                   sep = "\t", header = TRUE, stringsAsFactors = FALSE)

In [14]:
dim(ICGC)

[1] 60603   200

In [15]:
ICGC[1:10,1:10]

,X,H049.0CWK.genes.results,H049.7SD3.genes.results,H049.FG38.genes.results,H049.GP41.genes.results,H049.GWNW.genes.results,H049.JVCT.genes.results,H049.PF1J.genes.results,H049.UV6K.genes.results,H049.WB9V.genes.results
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000000003.14,15.40,140.10,52.82,129.55,47.64,36.95,121.41,18.92,41.05
2,ENSG00000000005.6,0.21,62.17,1.64,0.56,2.23,0.09,21.33,0.32,0.13
3,ENSG00000000419.12,17.16,58.37,59.93,48.66,42.03,37.33,62.16,49.42,52.16
4,ENSG00000000457.14,1.20,10.51,4.88,6.52,3.05,2.62,10.15,5.42,5.53
5,ENSG00000000460.17,2.68,5.13,2.34,5.90,4.36,4.52,4.10,5.26,8.46
6,ENSG00000000938.13,3.35,27.88,1.72,2.92,2.53,2.18,5.13,7.28,3.80
7,ENSG00000000971.15,1.51,35.69,1.83,22.47,3.64,8.30,33.33,32.92,9.21
8,ENSG00000001036.13,16.44,92.66,36.06,64.75,120.05,15.22,77.51,65.97,57.51
9,ENSG00000001084.13,4.45,21.49,9.04,13.27,6.64,5.30,12.86,11.51,11.68


In [16]:
colnames(ICGC)[1] <- "ensembl_id"

Some cleanup for colnames

In [17]:
colnames(ICGC) <- gsub(".genes.results", "", colnames(ICGC))
colnames(ICGC) <- gsub("[.]", "-", colnames(ICGC))

In [18]:
dim(ICGC)

[1] 60603   200

In [19]:
head(ICGC)

,ensembl_id,H049-0CWK,H049-7SD3,H049-FG38,H049-GP41,H049-GWNW,H049-JVCT,H049-PF1J,H049-UV6K,H049-WB9V,⋯,ICGC_PA89,ICGC_PA9,ICGC_PA91,ICGC_PA92,ICGC_PA93,ICGC_PA94,ICGC_PA95,ICGC_PA96,ICGC_PA97,ICGC_PA99
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000000003.14,15.40,140.10,52.82,129.55,47.64,36.95,121.41,18.92,41.05,⋯,10.59,13.17,5.95,7.56,12.42,11.44,12.69,9.92,12.44,10.51
2,ENSG00000000005.6,0.21,62.17,1.64,0.56,2.23,0.09,21.33,0.32,0.13,⋯,0.05,0.01,0.09,0.03,0.04,0.06,0.13,0.02,0.07,0.03
3,ENSG00000000419.12,17.16,58.37,59.93,48.66,42.03,37.33,62.16,49.42,52.16,⋯,5.83,3.99,4.26,4.41,4.03,4.62,6.50,6.17,6.01,5.47
4,ENSG00000000457.14,1.20,10.51,4.88,6.52,3.05,2.62,10.15,5.42,5.53,⋯,1.88,1.11,1.27,1.33,1.41,1.73,1.81,1.81,2.19,1.83
5,ENSG00000000460.17,2.68,5.13,2.34,5.90,4.36,4.52,4.10,5.26,8.46,⋯,0.68,1.02,0.91,0.63,1.05,1.06,1.38,1.19,1.10,1.37
6,ENSG00000000938.13,3.35,27.88,1.72,2.92,2.53,2.18,5.13,7.28,3.80,⋯,0.58,0.45,0.54,0.55,0.72,0.48,0.41,0.88,1.22,0.65


Remove PAR_Y pseudoautosomal regions

In [20]:
ICGC <- ICGC[!grepl("PAR_Y", ICGC$ensembl_id),]

In [21]:
dim(ICGC)

[1] 60558   200

In [22]:
ICGC$ensembl_id <- gsub("[.].*", "", ICGC$ensembl_id)

In [23]:
ICGC_dedup <- ICGC[ !duplicated(ICGC$ensembl_id),]

In [24]:
dim(ICGC_dedup)

[1] 60558   200

In [25]:
ICGC_exp_matrix_primary <- matchexp_matrixfx(ICGC_dedup, estimate_manifest_primary_clean,"ICGC", "Primary", "sample_id")

dimensions of exp_matrix: ICGC_dedup



[1] 60558   200


dimensions of estimate_manifest_df for: ICGC sample_type:Primary



[1] 196  21


dimensions of exp_matrix_matchingset for: ICGC_dedup with matching column: sample_id



[1] 60558   195


In [26]:
save(ICGC_exp_matrix_primary,
     file = paste0(datapath, "exp_mat/DKFZ_tpm_matrix_primary.RData"))

## TARGET

In [27]:
TARGET_tpm_matrix <- read.table(file = paste0(datapath,"exp_mat/NBL_tpm_matrix_hugo.txt"),
                                sep = "\t", header = TRUE, stringsAsFactors = FALSE, check.names = F)

In [28]:
dim(TARGET_tpm_matrix)

[1] 60498   159

In [29]:
TARGET_tpm_matrix_primary <- matchexp_matrixfx(TARGET_tpm_matrix, estimate_manifest_primary_clean,
                                              "TARGET", "Primary", "sample_id")

dimensions of exp_matrix: TARGET_tpm_matrix



[1] 60498   159


dimensions of estimate_manifest_df for: TARGET sample_type:Primary



[1] 149  21


dimensions of exp_matrix_matchingset for: TARGET_tpm_matrix with matching column: sample_id



[1] 60498   149


Remove duplicated genes

In [30]:
TARGET_tpm_matrix_primary$ensembl <- gsub("[.].*", "", TARGET_tpm_matrix_primary$ensembl)

In [31]:
TARGET_tpm_matrix_primary_dedup <- TARGET_tpm_matrix_primary[!duplicated(TARGET_tpm_matrix_primary$ensembl),]

In [32]:
dim(TARGET_tpm_matrix_primary_dedup)

[1] 60498   151

In [33]:
save(TARGET_tpm_matrix_primary_dedup,
         file = paste0(datapath, "exp_mat/TARGET_tpm_matrix_primary_dedup.RData"))

NBL matrix hgnc only

In [34]:
TARGET_tpm_matrix_primary_dedup_hgnc <- TARGET_tpm_matrix_primary[!duplicated(TARGET_tpm_matrix_primary$gene_symbol),]
rownames(TARGET_tpm_matrix_primary_dedup_hgnc) <- TARGET_tpm_matrix_primary_dedup_hgnc$gene_symbol
TARGET_tpm_matrix_primary_dedup_hgnc$gene_symbol <- NULL
TARGET_tpm_matrix_primary_dedup_hgnc$ensembl <- NULL

In [35]:
head(TARGET_tpm_matrix_primary_dedup_hgnc)

,TARGET-30-PAPZYP-01A,TARGET-30-PASWFB-01A,TARGET-30-PATINJ-01A,TARGET-30-PARZCJ-01A,TARGET-30-PASNZU-01A,TARGET-30-PASHFA-01A,TARGET-30-PATAYJ-01A,TARGET-30-PAPBGH-01A,TARGET-30-PARFWB-01A,TARGET-30-PASNEF-01A,⋯,TARGET-30-PANUKV-01A,TARGET-30-PASJZC-01A,TARGET-30-PASGUT-01A,TARGET-30-PATEKG-01A,TARGET-30-PAPUEB-01A,TARGET-30-PANLET-01A,TARGET-30-PANZRV-01A,TARGET-30-PASXHE-01A,TARGET-30-PAIPGU-01A,TARGET-30-PASXIE-01A
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TSPAN6,16.92,19.52,19.61,12.02,25.68,17.70,34.44,28.97,9.02,22.57,⋯,5.35,32.39,24.13,15.12,18.45,22.95,24.86,6.48,9.14,10.25
TNMD,1.01,0.05,0.03,0.07,0.05,0.11,0.22,0.00,0.04,0.20,⋯,0.23,0.05,0.05,0.24,0.16,0.04,0.02,0.17,0.08,0.06
DPM1,30.13,34.49,34.82,31.81,30.77,27.38,34.30,31.66,52.27,42.03,⋯,39.23,38.74,32.44,31.46,32.12,36.87,36.54,50.94,31.96,32.82
SCYL3,3.85,6.99,3.85,7.51,5.20,2.95,9.87,6.04,6.46,4.48,⋯,3.84,12.27,6.70,3.54,6.74,7.25,13.12,7.12,1.78,3.59
C1orf112,5.72,8.02,6.17,5.99,5.67,3.87,9.78,8.76,10.83,5.50,⋯,10.08,7.07,6.39,2.26,9.90,8.66,17.77,9.05,3.12,7.29
FGR,6.12,0.69,1.98,23.08,0.91,2.86,7.10,1.11,6.53,3.66,⋯,0.52,4.11,4.59,4.84,15.40,4.99,2.92,4.38,4.70,0.43


In [36]:
write.table(TARGET_tpm_matrix_primary_dedup_hgnc,
                file = paste0(datapath, "exp_mat/tpm_matrix_ped_TARGET_NBL_HGNConly.txt"),
                sep = "\t", quote = F, row.names = T)

## TCGA

In [37]:
TCGA_tpm_matrix <- read.table(file = paste0(datapath,"exp_mat/TCGA_tpm_matrix_hugo.txt"),
                              sep = "\t", header = TRUE, stringsAsFactors = FALSE, check.names = F)

In [38]:
head(TCGA_tpm_matrix)

,ensembl,gene_symbol,TCGA-29-1781-01A,TCGA-57-1585-01A,TCGA-13-1488-01A,TCGA-10-0928-01A,TCGA-30-1857-01A,TCGA-13-1507-01A,TCGA-25-1623-01A,TCGA-24-1423-01A,⋯,TCGA-06-0158-01A,TCGA-06-0211-01A,TCGA-06-0680-11A,TCGA-06-0157-01A,TCGA-14-0817-01A,TCGA-02-2485-01A,TCGA-06-0190-02A,TCGA-02-0047-01A,TCGA-27-2526-01A,TCGA-28-5207-01A
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000000003.14,TSPAN6,29.58,24.50,19.06,47.26,19.73,17.41,19.54,28.56,⋯,82.59,55.08,8.92,94.90,77.21,72.05,33.24,44.36,64.16,66.87
2,ENSG00000000005.5,TNMD,0.02,1.15,1.36,0.34,0.48,0.15,0.12,0.07,⋯,0.25,0.27,0.10,0.13,2.39,0.11,0.51,0.11,0.67,2.39
3,ENSG00000000419.12,DPM1,129.16,42.98,29.34,18.92,35.28,35.98,87.04,58.26,⋯,33.94,37.36,27.62,33.03,44.02,36.17,40.78,25.55,41.58,44.49
4,ENSG00000000457.13,SCYL3,4.15,3.22,2.14,3.15,3.18,2.87,3.77,5.46,⋯,5.09,4.46,4.58,3.67,6.17,5.35,3.36,4.21,1.77,7.44
5,ENSG00000000460.16,C1orf112,5.76,3.91,2.98,5.51,2.55,3.47,5.94,5.32,⋯,6.42,6.73,1.26,6.47,8.16,8.57,6.91,4.78,3.09,4.64
6,ENSG00000000938.12,FGR,5.41,5.04,1.94,0.94,3.26,2.77,6.86,1.82,⋯,7.79,11.29,2.92,5.16,5.79,6.92,7.21,9.74,7.13,12.78


In [39]:
TCGA_tpm_matrix_primary <- matchexp_matrixfx(TCGA_tpm_matrix, estimate_manifest_primary_clean,"TCGA", "Primary", "sample_id")

dimensions of exp_matrix: TCGA_tpm_matrix



[1] 60498  3036


dimensions of estimate_manifest_df for: TCGA sample_type:Primary



[1] 2452   21


dimensions of exp_matrix_matchingset for: TCGA_tpm_matrix with matching column: sample_id



[1] 60498  2452


Remove duplicated ensembl id (if any)

In [40]:
TCGA_tpm_matrix_primary$ensembl <- gsub("[.].*", "", TCGA_tpm_matrix_primary$ensembl)

In [41]:
TCGA_tpm_matrix_primary_dedup <- TCGA_tpm_matrix_primary[!duplicated(TCGA_tpm_matrix_primary$ensembl),]

In [42]:
dim(TCGA_tpm_matrix_primary_dedup)

[1] 60498  2454

In [43]:
save(TCGA_tpm_matrix_primary_dedup,
       file = paste0(datapath, "exp_mat/TCGA_tpm_matrix_primary_dedup.RData"))

## Merge all

In [44]:
colnames(TARGET_tpm_matrix_primary_dedup)[1:2] 
colnames(TCGA_tpm_matrix_primary_dedup)[1:2] 
colnames(ICGC_exp_matrix_primary)[1:2] 
colnames(CBTN_tpm_matrix_primary_dedup)[1:2]

[1] "ensembl"     "gene_symbol"

[1] "ensembl"     "gene_symbol"

[1] "ensembl_id" "H049-0CWK"

[1] "ensembl_id"  "hgnc_symbol"

In [45]:
colnames(TARGET_tpm_matrix_primary_dedup)[1]  <- "ensembl_id"
colnames(TCGA_tpm_matrix_primary_dedup)[1]  <- "ensembl_id"

merge with ensembl ids

In [46]:
tpm_matrix_ped <- merge(CBTN_tpm_matrix_primary_dedup, TARGET_tpm_matrix_primary_dedup, by = "ensembl_id")

In [47]:
dim(CBTN_tpm_matrix_primary_dedup)
dim(TARGET_tpm_matrix_primary_dedup)
dim(tpm_matrix_ped)

[1] 56505   634

[1] 60498   151

[1] 55391   784

In [48]:
tpm_matrix_ped <- merge(tpm_matrix_ped, ICGC_exp_matrix_primary, by = "ensembl_id")

In [49]:
dim(ICGC_exp_matrix_primary)
dim(tpm_matrix_ped)

[1] 60558   197

[1] 54969   980

In [50]:
head(tpm_matrix_ped)

,ensembl_id,hgnc_symbol,7316-1926,7316-489,7316-13,7316-3936,7316-873,7316-2669,7316-1928,7316-1655,⋯,ICGC_PA89,ICGC_PA9,ICGC_PA91,ICGC_PA92,ICGC_PA93,ICGC_PA94,ICGC_PA95,ICGC_PA96,ICGC_PA97,ICGC_PA99
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,ENSG00000000003,TSPAN6,8.66,6.15,0.73,1.50,29.41,4.62,6.65,1.78,⋯,10.59,13.17,5.95,7.56,12.42,11.44,12.69,9.92,12.44,10.51
2,ENSG00000000005,TNMD,0.02,0.03,0.02,0.01,0.33,0.03,0.07,0.01,⋯,0.05,0.01,0.09,0.03,0.04,0.06,0.13,0.02,0.07,0.03
3,ENSG00000000419,DPM1,6.15,5.09,1.09,3.68,4.89,4.75,3.07,4.58,⋯,5.83,3.99,4.26,4.41,4.03,4.62,6.50,6.17,6.01,5.47
4,ENSG00000000457,SCYL3,1.75,1.71,0.44,1.44,1.52,1.61,1.01,1.18,⋯,1.88,1.11,1.27,1.33,1.41,1.73,1.81,1.81,2.19,1.83
5,ENSG00000000460,C1orf112,1.12,2.78,0.43,0.71,1.00,1.64,0.58,1.61,⋯,0.68,1.02,0.91,0.63,1.05,1.06,1.38,1.19,1.10,1.37
6,ENSG00000000938,FGR,0.90,0.18,0.36,1.32,0.93,1.01,0.83,0.09,⋯,0.58,0.45,0.54,0.55,0.72,0.48,0.41,0.88,1.22,0.65


In [51]:
dim(tpm_matrix_ped)

[1] 54969   980

In [52]:
rownames(tpm_matrix_ped) <- tpm_matrix_ped$ensembl_id

tpm_matrix_ped$gene_symbol <- NULL
tpm_matrix_ped$ensembl_id <- NULL

In [53]:
head(tpm_matrix_ped)

,hgnc_symbol,7316-1926,7316-489,7316-13,7316-3936,7316-873,7316-2669,7316-1928,7316-1655,7316-2251,⋯,ICGC_PA89,ICGC_PA9,ICGC_PA91,ICGC_PA92,ICGC_PA93,ICGC_PA94,ICGC_PA95,ICGC_PA96,ICGC_PA97,ICGC_PA99
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000000003,TSPAN6,8.66,6.15,0.73,1.50,29.41,4.62,6.65,1.78,23.07,⋯,10.59,13.17,5.95,7.56,12.42,11.44,12.69,9.92,12.44,10.51
ENSG00000000005,TNMD,0.02,0.03,0.02,0.01,0.33,0.03,0.07,0.01,0.02,⋯,0.05,0.01,0.09,0.03,0.04,0.06,0.13,0.02,0.07,0.03
ENSG00000000419,DPM1,6.15,5.09,1.09,3.68,4.89,4.75,3.07,4.58,5.40,⋯,5.83,3.99,4.26,4.41,4.03,4.62,6.50,6.17,6.01,5.47
ENSG00000000457,SCYL3,1.75,1.71,0.44,1.44,1.52,1.61,1.01,1.18,1.13,⋯,1.88,1.11,1.27,1.33,1.41,1.73,1.81,1.81,2.19,1.83
ENSG00000000460,C1orf112,1.12,2.78,0.43,0.71,1.00,1.64,0.58,1.61,0.74,⋯,0.68,1.02,0.91,0.63,1.05,1.06,1.38,1.19,1.10,1.37
ENSG00000000938,FGR,0.90,0.18,0.36,1.32,0.93,1.01,0.83,0.09,0.46,⋯,0.58,0.45,0.54,0.55,0.72,0.48,0.41,0.88,1.22,0.65


remove non-coding-RNA and pseudogenes

In [54]:
hg38 <- useMart(biomart="ENSEMBL_MART_ENSEMBL", host="www.ensembl.org", 
                path="/biomart/martservice", dataset="hsapiens_gene_ensembl")

In [55]:
ensembls <- rownames(tpm_matrix_ped)

In [56]:
ensembl_hgnc_type <- getBM(filters="ensembl_gene_id", 
                           attributes=c("hgnc_symbol","ensembl_gene_id", "entrezgene_id", "gene_biotype"), 
                           values= ensembls, mart=hg38)

In [57]:
table(ensembl_hgnc_type$gene_biotype, useNA = "always")


                         IG_C_gene                    IG_C_pseudogene 
                                14                                  9 
                         IG_D_gene                          IG_J_gene 
                                37                                 18 
                   IG_J_pseudogene                          IG_V_gene 
                                 3                                145 
                   IG_V_pseudogene                             lncRNA 
                               181                              14116 
                             miRNA                           misc_RNA 
                              1447                               1074 
                           Mt_rRNA                            Mt_tRNA 
                                 2                                 22 
            polymorphic_pseudogene               processed_pseudogene 
                                44                              10079 
     

In [58]:
biotypes <- as.data.frame(table(ensembl_hgnc_type$gene_biotype), stringsAsFactors = F)$Var1

In [59]:
biotypes_pseudogenes <- biotypes[grepl("pseudo", biotypes)]
biotypes_RNA <- biotypes[grepl("RNA", biotypes)]

In [60]:
biotypes_RNA

[1] "lncRNA"          "miRNA"           "misc_RNA"        "Mt_rRNA"        
 [5] "Mt_tRNA"         "rRNA"            "rRNA_pseudogene" "scaRNA"         
 [9] "scRNA"           "snoRNA"          "snRNA"           "sRNA"           
[13] "vault_RNA"

In [61]:
biotypes_pseudogenes

[1] "IG_C_pseudogene"                    "IG_J_pseudogene"                   
 [3] "IG_V_pseudogene"                    "polymorphic_pseudogene"            
 [5] "processed_pseudogene"               "pseudogene"                        
 [7] "rRNA_pseudogene"                    "TR_J_pseudogene"                   
 [9] "TR_V_pseudogene"                    "transcribed_processed_pseudogene"  
[11] "transcribed_unitary_pseudogene"     "transcribed_unprocessed_pseudogene"
[13] "translated_processed_pseudogene"    "translated_unprocessed_pseudogene" 
[15] "unitary_pseudogene"                 "unprocessed_pseudogene"

In [62]:
rm_genes <- ensembl_hgnc_type$ensembl_gene_id[ensembl_hgnc_type$gene_biotype %in% c(biotypes_RNA,"TEC", biotypes_pseudogenes)]

In [63]:
dim(tpm_matrix_ped)

[1] 54969   978

In [64]:
tpm_matrix_ped <- tpm_matrix_ped[!rownames(tpm_matrix_ped) %in% rm_genes,]

In [65]:
dim(tpm_matrix_ped)

[1] 20105   978

In [66]:
tpm_matrix_ped_ensembl <- tpm_matrix_ped[, colnames(tpm_matrix_ped) != "hgnc_symbol"]

In [67]:
head(tpm_matrix_ped_ensembl)

,7316-1926,7316-489,7316-13,7316-3936,7316-873,7316-2669,7316-1928,7316-1655,7316-2251,7316-2622,⋯,ICGC_PA89,ICGC_PA9,ICGC_PA91,ICGC_PA92,ICGC_PA93,ICGC_PA94,ICGC_PA95,ICGC_PA96,ICGC_PA97,ICGC_PA99
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ENSG00000000003,8.66,6.15,0.73,1.50,29.41,4.62,6.65,1.78,23.07,0.82,⋯,10.59,13.17,5.95,7.56,12.42,11.44,12.69,9.92,12.44,10.51
ENSG00000000005,0.02,0.03,0.02,0.01,0.33,0.03,0.07,0.01,0.02,0.00,⋯,0.05,0.01,0.09,0.03,0.04,0.06,0.13,0.02,0.07,0.03
ENSG00000000419,6.15,5.09,1.09,3.68,4.89,4.75,3.07,4.58,5.40,0.78,⋯,5.83,3.99,4.26,4.41,4.03,4.62,6.50,6.17,6.01,5.47
ENSG00000000457,1.75,1.71,0.44,1.44,1.52,1.61,1.01,1.18,1.13,0.13,⋯,1.88,1.11,1.27,1.33,1.41,1.73,1.81,1.81,2.19,1.83
ENSG00000000460,1.12,2.78,0.43,0.71,1.00,1.64,0.58,1.61,0.74,0.25,⋯,0.68,1.02,0.91,0.63,1.05,1.06,1.38,1.19,1.10,1.37
ENSG00000000938,0.90,0.18,0.36,1.32,0.93,1.01,0.83,0.09,0.46,0.40,⋯,0.58,0.45,0.54,0.55,0.72,0.48,0.41,0.88,1.22,0.65


In [68]:
save(tpm_matrix_ped_ensembl,
         file = paste0(datapath, "exp_mat/tpm_matrix_ped_primary_dedup_ensembl.RData"))  

In [69]:
rownames(tpm_matrix_ped) <- tpm_matrix_ped$hgnc_symbol
tpm_matrix_ped$hgnc_symbol <- NULL

In [70]:
head(tpm_matrix_ped)

,7316-1926,7316-489,7316-13,7316-3936,7316-873,7316-2669,7316-1928,7316-1655,7316-2251,7316-2622,⋯,ICGC_PA89,ICGC_PA9,ICGC_PA91,ICGC_PA92,ICGC_PA93,ICGC_PA94,ICGC_PA95,ICGC_PA96,ICGC_PA97,ICGC_PA99
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TSPAN6,8.66,6.15,0.73,1.50,29.41,4.62,6.65,1.78,23.07,0.82,⋯,10.59,13.17,5.95,7.56,12.42,11.44,12.69,9.92,12.44,10.51
TNMD,0.02,0.03,0.02,0.01,0.33,0.03,0.07,0.01,0.02,0.00,⋯,0.05,0.01,0.09,0.03,0.04,0.06,0.13,0.02,0.07,0.03
DPM1,6.15,5.09,1.09,3.68,4.89,4.75,3.07,4.58,5.40,0.78,⋯,5.83,3.99,4.26,4.41,4.03,4.62,6.50,6.17,6.01,5.47
SCYL3,1.75,1.71,0.44,1.44,1.52,1.61,1.01,1.18,1.13,0.13,⋯,1.88,1.11,1.27,1.33,1.41,1.73,1.81,1.81,2.19,1.83
C1orf112,1.12,2.78,0.43,0.71,1.00,1.64,0.58,1.61,0.74,0.25,⋯,0.68,1.02,0.91,0.63,1.05,1.06,1.38,1.19,1.10,1.37
FGR,0.90,0.18,0.36,1.32,0.93,1.01,0.83,0.09,0.46,0.40,⋯,0.58,0.45,0.54,0.55,0.72,0.48,0.41,0.88,1.22,0.65


In [71]:
dim(tpm_matrix_ped)

[1] 20105   977

In [72]:
save(tpm_matrix_ped,
     file = paste0(datapath, "exp_mat/tpm_matrix_ped_primary_dedup.RData"))   

In [73]:
write.table(tpm_matrix_ped,
            file = paste0(datapath, "exp_mat/tpm_matrix_ped_primary_dedup.txt"), sep = "\t", quote = F)